# Ensemble Learrning and Random Forest

- Ensemble learning is when you aggregate the predictions oof a group of predictors like c;assifiersn and regressors you will often get a better rprediction with this than the ebst individual predictor 
- a grroup of prdictors is called an ensembklle so so this technique is called ensemble learning and the algorithm is called an ensemple ,ethjd
- for example you can traing a ggroup of decision tree classifiers on a different random subset of the training set . To make predictions yoiu get all the predictions of the indiivual trees then predict the class that getrrs the most votes .
- the ensemblle of decision trees is called a reandom forest this is one of the most powerful machine learning algorrithms
- so ensemble methods are used at the endd of a project when you built a few good predictors to combvine them into a better predictor 


#### vOTING Classifiers
- if we tain a load of classifiers and they all have a good accurrac a siumpkle way to get an even betterr classifier is to aggregate the predictions of each classifier and prrediuct the class that get the most vots
- the majorit botring classifier is called a hard voting classifier
- this votriung clasifier is higher accuracy because of the lalw of larrhe if we had 1000 classifierrs that are onl correct 51 percent of the time so barel better that random huessing iofg we prerdict the ,ajoritty vvoted class we can expect 75 perrcent accuracy.
- this is tue io all the classifierrrs are perfectl independednt making uncorrelated errorrrsbut in reality they wont be perfect; independent
- ensemble methods work best when the predictors are as independent from one another as  possible for exmaple trainign them, using vbery differrent algorithms so they all make difffernt types of erros improvingoveralll esemble acduracy
8

#### Lets train an ensemble

In [1]:
#Moons Data

from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

X,y= make_moons(n_samples=1000,noise=0.6)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
#check accurracy
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))


LogisticRegression 0.775
RandomForestClassifier 0.765
SVC 0.775
VotingClassifier 0.795


- Ass you can see the ensemble voting classifier is the most accurate
- soft votiung is when is the classifier provide sprobability estimates such as the SVC class we can switch toi oft voting which is when the predictions are weighted by the probabilit esditmnates 

#### Baggin and pasting
- we cxan het a diverse set of classifier using different trrainign algorithms
- another apprreach is to use the same training algotith for every prredictor and train them on different random subnsets of the trainign set 
- when the sampling is performed iwht replacement this method is called baggin 
-- if there is no replacement itnis called pasting
- both baggin and pasig allow training instances to sbe sampkled sevberal tims across multiple predicots biut l baggin allopwas training instances to be samples several times for the same prredictorr 
- once all the rrpedictorrs are trained the ensemnle can ,ake a prediction for a new instance by aggrregating the prediciton pf all rpedictorr s
- the aggrrergation fuincvtion is useually the statisitcal mode ioe the most frewuent prrediction or the average for regresiiom
- each individual prredictor has a higher bnias than were trained on the rginal trraining set but aggrergation reduces bnoth bias and variance 
- ensemble has a similarr bias but lower variance than a simgle predictorr trained on the orrignal trrainin set
- baggin and pastin scale very well


# baggin and pasting in Sklearn
- sklearn offers an API for baggin asn pasting with the `BaggingClassifier` class or `BaggingRegression` for regrerssion.The code trains on an ensemble of 500 Decision Tree classifierrs each trained on a 1== trrainign instances randoml;y sampled from the training set with replacement ie baggin
- if you want to use pasting set `bootstrap=False`
- The n_jobs paraemter telll sklearn the numbner of CPU cores to use




In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    max_samples=100,bootstrap=True,n_jobs=-1)
    
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

Bootstrapping  introfduced more diverrsity in the sibsets that each orredictor ins trained on. so baggin has a slightl hier bias than asting buit the extra diversity ,eanms they are less correellated ,e,and the ensemble variance is refuce d
baggin oftern rresults in better models but if there is m,ore time and CPU [
opwer oi can use cross validation to check borth methods

#### Out of bag evaluation
the oob instances are the data instances that are not sampled at all with replacments 
- a bangin ensemple can be evaluate siusing oob instances wihtout the need for a serpate validation set so of there are lot of estimators the each instance of ahte traning set will likely be an oob instance ovf several extimators so these estimatorrs can be used to make afair exembke rpediction forr that instance
- we can do this b setting `oob_score = True` by creating a BaggingClassifier to request an automatic oob evaluation after trraining,,
The foloowing code demostrates this


In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),n_estimators=500,
    bootstrap=True,n_jobs=-1,oob_score=True)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.76

This means the baggin classifer is likel to achieve round 75 percent on the test set from validating o hte oobs

In [6]:
from sklearn.metrics import accuracy_score
pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.765

#### Random Forests
This is an ensemble of Decition Trees which arre generallyy trained with the baggin method with the max_samples set to the size of the trainign set
0 we can esilt implement this isogm the randomforest classifier class instead of crreating a baggin classifier and passing decisiont rees 

In [7]:
from sklearn.ensemble import RandomForestClassifier


rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred_rf = rnd_clf.predict(X_test)

The RANDOM FOREST CLASSIFER has all the hyper params of the decvision trree classifier and the bagign classifeir
- the alfotihm introduces extera randomess when grwing trees . it doesnt search for the est features when splitting nodes it seaches for the ebst feature amond a random subset of feateues 
- this fgive betterr tree diversity and trades higher bias for a lower variance

#### Ectra Trees
- this is when ou make the tress more random by using random thrreshold dfor reach dfeature trather thans earrhing for the best possible threshold 
- we call theis ectrremely randomised trees 
- this is much gadter than a nojmral random forest 
- call the `ExtraTreesClassifier`
- usuall you test bnoth random forests und ciomoare them using cross validation and fgrid seaarch to determin whichn is better


#### Random forests can show featurre importance 
- sklearn measures a featruer imporance by looking how much thrrre tree nodes that use that featurrre reduce impurit on averrage across all trees in the forest ,, ie its a weigherd abergae where each nodes weight is equal tot he number of trainign samples thhat are associate with it 
- sklearn comuites this score automaticall afeter trrainign the scaled is so the sum is 1 

we do this below

In [8]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500,n_jobs=-1)
rnd_clf.fit(iris['data'],iris['target'])
for name, score in zip(iris['feature_names'],rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.09859282149466389
sepal width (cm) 0.023595486055755205
petal length (cm) 0.428483564010211
petal width (cm) 0.44932812843937


You can see clearly the peral legnth and width are the most important features

#### Boosting
- This is any ensemble method that cominies several weak learners intoa strong learner
- The gerna;l idea is to train predictors sequentiall, each try to correct its predecessor
- The most po;ul;at are Ada Boost and Gradient Boosting


#### AdaBoost
one way fo the new predictor to correct its predecessor is tio pay a bit more attention to the training instances that the predeccsirrre undersfitted 
- this results in a enw predictors focusing morre and more on the ahrrd cases 
- this is the technique use dby ADA boost 
- when trianign an ADABoost classifier thenalforith tains a base classfier like a decision tree and uses it to make prediction on the trraining set 
- the algo then increraes the relativwe weight of the misclassified training instances
- it now uses a second classifier useing the iopdated trainign weight and make new predictions-
- this now cares on

- lets say the first classifier gets many instacnes wrong so teir weights get boosted the secoind classifier does a better job on these instance ans so on 

- its like gradient decent but it addes predictors to the esembkle making it better ather than minimuising a cost function by teweaking params

- once all the predictors are trained the ensembkle makes predictions like bagiin orr pasting but the predictors havb different weigfhts depending on their overall accuracy on the weighted trraining set 
- uncoftunatel as its sequential learrning it cannot be pareelerised since reach prredictor strianaes in sequence so clasing i not as good as baaggin or pasting
- Skleasrn used a multioclass version of AdaBoost called SAMME if the perdicoprs c can estimate class probs we can use SAMME.R which is ghenerall better 
- tghe code beow traines the AdaBoost classifier using 200 decisoin stuimps shich is a decision tree iwth a max depth of 1 m, this is aso the defauilt base estimator for ther ada boos classifier

In [9]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),n_estimators=200,
    algorithm='SAMME.R',learning_rate=0.5) #algorithmmis the boosting method for weak learners
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

#### Gradient Boosting

-Grradient boosting works by sequentially adding predictors to the ensemble each one correcting its predessor but instead of tweaking the instance weights at every iteration like Ada it tries to fit a new prredictor to the rrersidual erros made b the pevious prredictor 
if we use decision trees and the base we can call it gradient rree boosting orr grradient boosted regression trees 
- lets fit a decision tree regressor to the training set

In [10]:
#EXAMPLE CODE FOR SYNTAX
from sklearn.tree import DecisionTreeRegressor 

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X_train,y_train)
#now train a second decision tree on the rrecidual eros made bt the firs predictor

y2 = y_train - tree_reg1.predict(X_train)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train,y2)

# now we train a thirrd regrssor on the seond residuals 
y3 = y2 - tree_reg2.predict(X_train)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X_train,y3)

# now we have an esemblke conatining three trees it can make predictions 
#on a new instance b additng ip the rpedcition of all the trees

y_pred = sum(tree.predict(X_test) for tree in (tree_reg1,tree_reg2,tree_reg3)) 
## we sum as the models adjusting the resiudals for places wherre it perfformed badly


wE Can implement the gradient boosint rregressorr quinck;y using the code below


In [13]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1, max_depth=2, n_estimators=3)

in order to diuns the optimal number of trrees ou can use earrl stopping using the `staged_predict()` method
- it returns an iterrator over the precitions mad ebyt the ensembnle at each staghe of training ie with one tree two trees etc 
- the following code trrains the GBRRT ensemble to fin the optimal number of trees then train=s another GBRRT using the optimal nubner of rtrees


```import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)


errors = [mean_squared_error(y_train,y_pred)
         for y_prerd in gbrt.staged_predict(X_test)]
bst_n_estimators = np.argmin(errors)+1
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,Y_train)```



#### Stacking
- this is the case when instead of using functions such as hard or soft voting we can acutallly tain a model to perform the aggrreghating 
- the dinal predictor is called the blender or meta learner
- to trin the blender a commoon aprroach is to use a holld out set 
- the training set is spliot intop two subsets the firrst subset is used to tain predictorrs in the firs tlayer 
- the firtst layer predictorrs are now iused to maek mprredictions on the second holdout set 
- this ensures th predictions are clean and since the predictos never saw these isntances during tainign each instance in the hold out set there arre now predictions for each predictor used 
- we can crreate a new training set using thes predictesd values as input ffeatures and keeping the same starrget calues 
- the blender is trained on this new trainin set so it learns to predict the tarrrget calue fiven the first layerrs predictions
- we can furthe rthis by acxtuall training several blenders using different predictors to get a whole layer of blanders - sklearn does nt support stacking directl but it can be implemented 